# Feature Engineering

In [1]:
import os
import re
import statsmodels.api as sm
import sys
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import PolynomialFeatures

%matplotlib inline
warnings.filterwarnings('ignore')

In D:\Users\Marcelo\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In D:\Users\Marcelo\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In D:\Users\Marcelo\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In D:\Users\Marcelo\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In D:\Users\Marcelo\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classi

In [2]:
sys.path.append(os.path.join('..', 'src'))

In [3]:
import importlib
import utils, params
importlib.reload(utils)
from utils import build_data_dict

importlib.reload(params)
from params import ProjectParameters

# define functions 

In [4]:
def build_polynomials(df, cols, method = 'simple_square', degrees=2):  
    print('number of columns before building polynomials:', df.shape[1])
    if method == 'simple_square':
        poly = df[cols]**2
        poly.columns = [c+'_power2' for c in df[cols].columns]
        df = pd.concat([df, poly], axis=1)
    elif method == 'k_degree':
        poly = PolynomialFeatures(2, include_bias = False)
        transformed = poly.fit_transform(df[cols])
        expanded_cols = poly.get_feature_names(df.columns)
        df = pd.DataFrame(transformed, columns = expanded_cols)
    print('number of columns after building polynomials:', df.shape[1])
    
    return df

def get_distribution(column, save_file):
    #Check the new distribution 
    sns.distplot(column, fit=norm);

    # Get the fitted parameters used by the function
    (mu, sigma) = norm.fit(column)

    # plot the distribution
    plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
                loc='best')
    plt.ylabel('Frequency')
    plt.title('estimate distribution')
    plt.savefig(os.path.join(reports,save_file+'.jpg'), bbox_inches = "tight")
    plt.close()
    
    fig = plt.figure()
    res = stats.probplot(column, plot=plt)
    plt.savefig(os.path.join(reports,save_file+'_regline.jpg'), bbox_inches = "tight")
    plt.close()

### functions to deal with skewness
* bc_lambda is the box cox transformation on the variables. bc_lambda = 0 is equivalent to log transformation.

In [5]:
def get_skewness(df):
    numeric_features = df.dtypes[(df.dtypes != "object") & (df.dtypes != "bool")].index.to_list()
    skewed_features = df[numeric_features].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
    skewness = pd.DataFrame({'skew' :skewed_features})
    
    return skewness

def treat_skewness(df, df_name, bc_lambda = 0):
    skewness = get_skewness(df)
    print(skewness.head(10))

    skewness = skewness[skewness['skew'] > 0.75]
    print("There are {} skewed features prone to Box Cox transform on {} data".format(skewness.shape[0], df_name))

    skewed_features = skewness.index

    print(df_name)
    for feature in skewed_features:
        df[feature] = boxcox1p(df[feature], bc_lambda)
        
    return df

# Define paths

In [6]:
inputs = os.path.join('..', 'data', '02_intermediate')
outputs = os.path.join('..', 'data', '03_processed')
reports = os.path.join('..', 'data', '06_reporting')

# Data capture

In [7]:
data_list = ['X_train', 'X_test', 'X_train_oh', 'X_test_oh', 'y_train', 'y_test']

dfs_dict = build_data_dict(inputs, data_list)
for df in data_list:
    print(dfs_dict[df].shape)

loading data into dictionary
(3500, 13)
(1500, 13)
(3500, 96)
(1500, 87)
(3500, 1)
(1500, 1)


In [8]:
for k,value in dfs_dict.items():
    print('shape of', k, 'is:', value.shape)

shape of X_train is: (3500, 13)
shape of X_test is: (1500, 13)
shape of X_train_oh is: (3500, 96)
shape of X_test_oh is: (1500, 87)
shape of y_train is: (3500, 1)
shape of y_test is: (1500, 1)


In [9]:
dfs_dict['X_train'].head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country
id,,,,,,,,,,,,,
1516,46,Private,HS-grad,9,Married-civ-spouse,Adm-clerical,Husband,White,Male,0,0,58,United-States
15516,62,Private,Some-college,10,Divorced,Adm-clerical,Unmarried,White,Female,0,0,30,United-States
13206,36,Local-gov,Masters,14,Married-civ-spouse,Prof-specialty,Wife,White,Female,0,0,60,United-States
14145,51,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States
6815,45,Private,HS-grad,9,Separated,Tech-support,Not-in-family,White,Female,0,0,26,United-States


In [10]:
dfs_dict['X_train'].describe()

,age,education_num,capital_gain,capital_loss,hours_per_week
count,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000
mean,38.585429,10.047143,1200.606857,83.913714,41.008286
std,13.478291,2.594167,8071.276206,397.451061,12.436139
min,17.000000,1.000000,0.000000,0.000000,1.000000
25%,28.000000,9.000000,0.000000,0.000000,40.000000
50%,37.000000,10.000000,0.000000,0.000000,40.000000
75%,48.000000,12.000000,0.000000,0.000000,45.000000
max,90.000000,16.000000,99999.000000,2824.000000,99.000000


In [11]:
dfs_dict['X_test'].describe()

,age,education_num,capital_gain,capital_loss,hours_per_week
count,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000
mean,39.279333,10.106667,965.782000,94.801333,39.912667
std,14.158918,2.462383,6297.705554,427.900936,11.813018
min,17.000000,1.000000,0.000000,0.000000,1.000000
25%,28.000000,9.000000,0.000000,0.000000,40.000000
50%,38.000000,10.000000,0.000000,0.000000,40.000000
75%,49.000000,12.000000,0.000000,0.000000,45.000000
max,90.000000,16.000000,99999.000000,4356.000000,99.000000


# define continuous variables

In [12]:
numerical_cols = ProjectParameters().numerical_cols

# transformations

### polynomial features
* We might build simple polynomials with the square of continuous data
* another approach would be by using all possibilities from a degree of k. sklearn ref application: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html

In [13]:
test = build_polynomials(dfs_dict['X_train'], numerical_cols, method = 'simple_square', degrees=2)

number of columns before building polynomials: 13
number of columns after building polynomials: 18


### symmetry

In [14]:
from scipy import stats
from scipy.stats import norm, skew
from scipy.special import boxcox1p, logit

# change skewness

Now let's see try to adjust skewness in the distribution.

In [15]:
transformed_dict = {}

### independent variables

In [16]:
boxcox_lambda = 0
for df_name in ['X_train', 'X_test']:
    transformed_dict[df_name] = treat_skewness(dfs_dict[df_name], df_name, boxcox_lambda)

                     skew
capital_gain    11.154191
capital_loss     4.644252
age              0.528399
hours_per_week   0.285269
education_num   -0.311962
There are 2 skewed features prone to Box Cox transform on X_train data
X_train
                     skew
capital_gain    13.339554
capital_loss     4.583976
age              0.546534
hours_per_week  -0.084335
education_num   -0.225374
There are 2 skewed features prone to Box Cox transform on X_test data
X_test


### see how skewness changed

In [17]:
print('skewness after transformation:\r\n')
for df_name in ['X_train', 'X_test']:
    print(get_skewness(transformed_dict[df_name]).head(10))

skewness after transformation:

                    skew
capital_loss    4.453400
capital_gain    2.948299
age             0.528399
hours_per_week  0.285269
education_num  -0.311962
                    skew
capital_loss    4.198849
capital_gain    3.000268
age             0.546534
hours_per_week -0.084335
education_num  -0.225374


# save data 

In [18]:
for df in ['X_train', 'X_test']:
    transformed_dict[df].to_csv(os.path.join(outputs, df+'.csv'))
    dfs_dict[df].to_csv(os.path.join(outputs, df+'_skewed.csv'))
for df in ['y_train', 'y_test']:
    dfs_dict[df].to_csv(os.path.join(outputs, df+'.csv'))